# Scraping journalists then using twint to get who they follow

In [1]:
import sys
import os
import time
import json

import nest_asyncio
nest_asyncio.apply()
import twint

sys.path.insert(1, '../')
from src.data import journalists as journos

### 1. Getting journalist twitter handles according to topic

In [3]:
journo_handles = journos.get_handles_by_keyword('cyber')
print(len(journo_handles))

12


In [4]:
print(journo_handles)

['jennystrasburg', 'dannsimmons', 'LeoKelion', 'gordoncorera', 'joetidy', '_lucyingham', 'dannyjpalmer', 'SophiaFurber', 'SCFGallagher', 'MsHannahMurphy', 'JesscaHaworth', 'Ad_Nauseum74']


### 2. Loop over journalists and get who they follow

In [5]:
following = {}

for username in journo_handles:
    twint.output.follows_list = []
    c = twint.Config()
    c.Username = username
    c.User_full = False
    c.Store_object = True
    c.Hide_output = True
    
    twint.run.Following(c)
    
    following[username] = twint.output.follows_list

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is n

In [44]:
total = 0
all_names = []
for key in following.keys():
    total += len(following[key])
    all_names.extend(following[key])
unique = len(set(all_names))

print('Total number of handles pulled: '+str(total))
print('Number of unique usernames: '+str(unique))

failed = []
for key in following.keys():
    if len(following[key])==0:
        print('Zero following in list for @'+key)
        failed.append(key)

Total number of handles pulled: 21804
Number of unique usernames: 18918


If some of the requests seem to have failed, can try them again:

In [30]:
for username in failed:
    twint.output.follows_list = []
    c = twint.Config()
    c.Username = username
    c.User_full = False
    c.Store_object = True
    c.Hide_output = True
    
    twint.run.Following(c)
    
    following[username] = twint.output.follows_list

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.feed:Follow:IndexError
CRITICAL:root:twint.feed:Follow:IndexError


Interestingly, most of the handles must appear only once, suggesting there are only a few users that are followed by more than one of the journalists in this list.

In [43]:
with open('../data/raw/cyber_journalist_friends.json', 'r') as file:
    following = json.load(file)

In [47]:
new_following = {}

for i, key in enumerate(following.keys()):
    new_following[i] = {'screen_name': key,
                        'friends': following[key]}

In [54]:
datastring = ''
for key in new_following.keys():
            datastring += json.dumps(new_following[key])
with open('../data/raw/cyber_journalist_friends_3.json', "w") as file:
            json.dump(datastring, file)

In [56]:
import pandas as pd

In [60]:
df_test = pd.DataFrame.from_dict(new_following, orient='index')

In [61]:
df_test.head()

,screen_name,friends
0,jennystrasburg,"[RobaHusseini, HashemOsseiran, liveanthony, El..."
1,dannsimmons,"[RoyalMailHelp, RoyalMail, MoonchikM, TheCyber..."
2,LeoKelion,"[samscam, Danny_Cortes95, jennifercobbe, samue..."
3,gordoncorera,"[emilyrauhala, AuroraIntel, cdketchie, charles..."
4,joetidy,"[spiesdontdie, GOllumfun, 1337core, Beatrix_vS..."


In [62]:
df_test.to_csv('../data/raw/cyber_journalist_friends.csv', index=False)

In [10]:
from collections import Counter

In [33]:
counted = Counter(all_names)

In [34]:
counted.most_common(50)

[('MalwareTechBlog', 9),
 ('briankrebs', 9),
 ('gcluley', 8),
 ('TaylorLorenz', 7),
 ('alexhern', 7),
 ('jimwaterson', 7),
 ('faisalislam', 7),
 ('BBCBreaking', 7),
 ('ProfWoodward', 7),
 ('DaveLeeFT', 7),
 ('TechCrunch', 7),
 ('ruskin147', 7),
 ('alexstamos', 7),
 ('jamesrbuk', 7),
 ('NCSC', 7),
 ('KimZetter', 7),
 ('GossiTheDog', 7),
 ('geoffwhite247', 7),
 ('bbclaurak', 7),
 ('mikko', 6),
 ('dnvolz', 6),
 ('elonmusk', 6),
 ('realDonaldTrump', 6),
 ('TheEconomist', 6),
 ('troyhunt', 6),
 ('jleyden', 6),
 ('DanRaywood', 6),
 ('oliviasolon', 6),
 ('WIRED', 6),
 ('drjessicabarker', 6),
 ('charlottejee', 6),
 ('campuscodi', 6),
 ('joetidy', 6),
 ('SecurityCharlie', 6),
 ('josephfcox', 6),
 ('carolecadwalla', 6),
 ('amolrajan', 6),
 ('SwiftOnSecurity', 6),
 ('DarkReading', 6),
 ('meghamohan', 6),
 ('e_kaspersky', 6),
 ('iblametom', 6),
 ('zackwhittaker', 6),
 ('Peston', 6),
 ('lorenzofb', 6),
 ('FSecure', 5),
 ('ericgeller', 5),
 ('kashhill', 5),
 ('rcallimachi', 5),
 ('noUpside', 5)]

range(0, 20)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
